# Splashtop Secure Workspace Automation
Splashtop Secure Workspace provides rich set of API with full automation capability of managing and working within the workspace. Since secure workspace provides IT admin and end user an all-in-one platform to access all their applications and data, it is important to be able to automate the provisioning of the workspace and the applications within the workspace. The automation capability provide IT admin, developers and end users the unprecedented flexibility to manage and work within the workspace, for example:
* IT admin can automate the provisioning of the workspace and applications to the end users
* Developers can automate and orchestrate their geo-distributed applications within single piece of code
* End users can automate their daily tasks within the workspace
* System admins can automate the network configuration within the workspace 


## Secure Workspace as Code (SWaC)
With the full automation capability, this notebook can fulfill the following example use cases without manual install any application or configuration/setup:
* Authenticate to your workspace
* Managed the workspace as admin for example provision an example restful API application to all users group
* Tunnel to the remote restful API application and invoke the API



## Prerequisites
* Assuming you have an Splashtop Secure Workspace account
* Python and Jupyter notebook installed 
* Generate Secure Workspace API key 
    - Login to Splashtop Business web console, go to Settings -> API Keys -> Generate API Key, copy the API key.
    - update the apikey.py file with the API key    

## Install Python Libraries

In [121]:

!pip install requests PyJWT


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


## Setup Variables
setup the following cell with variables for your environment

In [122]:
import os
import json
import subprocess
import requests
import time
from apikey import apikey

DEBUG = True
curr_path = os.getcwd()

# Variables - fill these in
# apikey from import
server_url = 'demo.stage.ztw.splashtop.com'
org_name = 'demo'
env = 'stage'
site_name = 'cloud-connector'
network_name = 'demo'
edge_name = 'US-WEST-1'
EXE_FILE = ''
ERROR_MESSAGE = ''

## Authentication and Access Tokens
Run the following cell to authenticate to the Secure Workspace and get auth tokens. There are 2 types of tokens:
* iam_token: used for secure workspace IAM operations, e.g. authentication, user management, group management, etc.
* sdp_token: used for secure workspace zero trust application management operations, e.g. application provisioning, client management, etc.

In [124]:
# Get SDP token
apikey_token = {"apikey": apikey}
response = requests.post(f"https://{server_url}/iam/v1/apikey/authenticate", json=apikey_token)
if response.status_code == 200 and response.json().get('code')==0:
        sdp_token_new = response.json().get('data').get('sdp_token')
        iam_token_new = response.json().get('data').get('iam_token')
        if DEBUG:
            print(f'sdp_iam_token.json: {response.json()}')
        if not sdp_token_new:
            ERROR_MESSAGE += "Error: sdp_token_new is empty, get sdp token failed \n"
else:
        ERROR_MESSAGE += f"Error: HTTP {response.status_code} - {response.text} \n"




sdp_iam_token.json: {'code': 0, 'message': 'success', 'data': {'iam_token': 'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhY3RpdmUiOnRydWUsImF1dG9fdW5sb2NrIjp0cnVlLCJjdXJyZW50X29yZ2FuaXphdGlvbl9pZCI6ImZkMWMzOTdjLWE2YTctNDNiMy04MWVmLWRmMjE1ZjBiODZmOSIsImVtYWlsIjoieWFubGluLndhbmdAc3BsYXNodG9wLmNvbSIsImV4cCI6MTY4Mzg3NzM3OSwiZnVsbF9uYW1lIjoiWWFubGluIFdhbmciLCJncm91cHMiOlt7Imdyb3VwX2lkIjoiNjJhYjY5MzctMDdkMC00NDNjLThhYTEtZmQzYzkyNzRlYTVjIiwibmFtZSI6Il9hbGx1c2VyIn0seyJncm91cF9pZCI6IjhmY2ExNzVhLWY2Y2MtNDhmYS1hOGQzLTUwOTFjMTFkMzZiZiIsIm5hbWUiOiJfb3JnYWRtaW4ifSx7Imdyb3VwX2lkIjoiYTQwMzFmODMtYzY4Ni00ZTg1LWI3MDMtMTg3OWE3Y2ZmZjVlIiwibmFtZSI6ImxhYiJ9LHsiZ3JvdXBfaWQiOiJiMjM5MGNiNi01ZjdmLTRlMzAtOTRkMy0yODMzNTk2M2VhMTkiLCJuYW1lIjoiZW1wbG95ZWVzIn1dLCJpYXQiOjE2ODM4NzY3NzksImlkIjoiNzk1OWExZjItNWQ1OS00ODBiLWEyYjYtYjViMGM2NzI5MjYzIiwiaW5pdF9wd2QiOiIiLCJsb2dpbl9uYW1lIjoieXdhbmdAZGVtby5zdGFnZS56dHcuc3BsYXNodG9wLmNvbSIsIm5lZWRfY3JlYXRlX3VzZXIiOnRydWUsIm5lZWRfbWZhX3NldHRpbmdzIjpmYWxzZSwibmVlZF9yZXNldF9wd2QiOmZhbHNlLC

## Get Organization From IAM Token
From the IAM token, we can get the organization information. The organization is the top level container of the secure workspace. All the users, groups, applications, etc. are under the organization. It provides multi-tenant capability for each workspace account. The organization id is used for all the secure workspace API calls.

In [125]:
import jwt
import json

# Base64 encoded JSON token
encoded_token = iam_token_new

# Decoding the JWT
decoded_token = jwt.decode(encoded_token, options={"verify_signature":False})

# Loading the payload into a JSON object
payload_json = json.loads(json.dumps(decoded_token))

# Extracting the organization ID
organization_id = payload_json['current_organization_id']


## Retrieve Group Information
From the organization id, we can retrieve all the groups within the organization. In the example, we will get information of a built-in group - "All User".

In [126]:
# Get group_id for all user group
headers = {"Accept": "application/json, text/plain, */*", "Authorization": f"Bearer {iam_token_new}", "Content-Type": "application/json;charset=UTF-8"}
response = requests.get(f"https://{server_url}/iam/v1/organization/{organization_id}/groups?page_size=99999", headers=headers)

group_id  = ''
if response.status_code == 200 and response.json().get('code')==0:
        # find the "All User" group_id
        group_id = response.json().get('data').get('list')[0].get('group_id')
elif response.status_code != 200:
        ERROR_MESSAGE += f"Error: HTTP {response.status_code} - {response.text} \n"
        print (ERROR_MESSAGE)
else:
        ERROR_MESSAGE += f"Error: {response.json().get('message')} \n" 
        print (ERROR_MESSAGE)       
print(f'group_id : {group_id }')


group_id : 62ab6937-07d0-443c-8aa1-fd3c9274ea5c


## Retrieve Application Information & Provision Application to "All User" Group
From the organization id, we can retrieve all the applications within the organization. In the example, we will get information of an example application - "ztw-restful-cloud", then provision the application to the "All User" group.

In [128]:
# find the ztw-restful-cloud application
headers = {"Accept": "application/json, text/plain, */*", "Authorization": f"Bearer {sdp_token_new}", "Content-Type": "application/json;charset=UTF-8"}
response = requests.get(f"https://{server_url}/ztna/v1/application?search=&order=&desc=false&org_id={organization_id}", headers=headers)

application = {} 
if response.status_code == 200 and response.json().get('code')==0:
        # find the "All User" group_id
        apps = response.json().get('data').get('list')
        for app in apps:
            if app.get('name') == 'ztw-restful-cloud':
                application = app
                break 
elif response.status_code != 200:
        ERROR_MESSAGE += f"Error: HTTP {response.status_code} - {response.text} \n"
        print (ERROR_MESSAGE)
else:
        ERROR_MESSAGE += f"Error: {response.json().get('message')} \n" 
        print (ERROR_MESSAGE)       
print(f'application: {application}')
application_id = application.get('id')

#update the application by assigning all users group to it

# Check if the application not have the group_id, append all users group to it
if group_id not in application.get('group_ids'):
        application.get('group_ids').append(group_id)


headers = {"Accept": "application/json, text/plain, */*", "Authorization": f"Bearer {sdp_token_new}", "Content-Type": "application/json;charset=UTF-8"}
response = requests.put(f"https://{server_url}/ztna/v1/application/{application_id}",  headers=headers, data=json.dumps(application))

if response.status_code == 200 and response.json().get('code')==0:
        print (f"Update Application Success: {response.json().get('message')} \n")
elif response.status_code != 200:
        ERROR_MESSAGE += f"Error: HTTP {response.status_code} - {response.text} \n"
        print (ERROR_MESSAGE)
else:
        ERROR_MESSAGE += f"Error: {response.json().get('message')} \n" 
        print (ERROR_MESSAGE)       



application: {'org_id': 'fd1c397c-a6a7-43b3-81ef-df215f0b86f9', 'id': '4faf8585-6c2f-404f-b545-a894102266d0', 'edge_id': '8fdfcf14-d6a6-4b86-af4a-69b0eae5ce50', 'site_id': '307eb6bc-7188-45f4-a574-e7f72dc4fbfe', 'site_name': 'cloud-k8s-connector', 'site_display_name': 'cloud-k8s-connector', 'conn_id': '', 'conn_status': 0, 'name': 'ztw-restful-cloud', 'display_name': 'ztw-restful-cloud', 'network_name': 'demo', 'group_ids': ['anonymous'], 'protocol': 'http', 'hosts': [{'host': 'ztenv-myrestful.default.svc.cluster.local', 'port': 8080}], 'host': 'ztenv-myrestful.default.svc.cluster.local', 'port': 8080, 'ports': [8080], 'fqdn': 'ngzhzjg1odut.app.e1-sanfrancisco-c1-usw2.stage.ztw.splashtop.com', 'ip': '', 'access_type': 'all', 'webgate_url': '', 'app_web_config': '{"host":"ztenv-myrestful.default.svc.cluster.local","port":8080,"via_private_network":true,"max_session_duration":28800,"interception_uri":[]}', 'icon': 'http', 'status': 2, 'tunnel_status': 0, 'launch_script_id': '', 'max_sess

## Automate Secure Workspace CLI installation
In order to connect to secure workspace remote applications from automation code, we need to install the secure workspace CLI. The following code will automate the installation of the CLI using brew, assuming you are using macOS.

In [130]:
#on MacOS install brew if not installed
! /bin/bash -c '$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)'

/bin/bash: #!/bin/bash: No such file or directory


In [131]:
#on MacOS brew add tap if not added
!brew tap SplashtopInc/workspace


In [86]:
#on MacOS brew install the secure workspace cli client
!brew install sdpc

==> Fetching dependencies for splashtopinc/workspace/sdpc: git
==> Fetching git
==> Downloading https://ghcr.io/v2/homebrew/core/git/manifests/2.40.1
######################################################################### 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/git/blobs/sha256:df2e65dfd20e76
==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sh
######################################################################### 100.0%
==> Fetching splashtopinc/workspace/sdpc
==> Downloading https://s3.us-west-2.amazonaws.com/splashlock-assets.splashshiel
######################################################################### 100.0%
==> Installing sdpc from splashtopinc/workspace
Please update to Xcode 14.3 (or delete it).
Xcode can be updated from the App Store.

==> Installing dependencies for splashtopinc/workspace/sdpc: git
==> Installing splashtopinc/workspace/sdpc dependency: git
==> Pouring git--2.40.1.arm64_ventura.bottle.tar.gz
🍺  /opt/homeb

## Automate Network Tunneling to the Remote Application

With the secure workspace CLI installed, we can automate the network tunneling to the remote application. In the example, we will tunnel to the remote restful API application and invoke the API. When rune this cell, please notice the notebook will prompt for your local os password in order to run sudo, then you should wait for the cli to establish the tunnel, if the connection established, you should see the output: 

```bash
ztw-restful-cloud connect success
```

In [132]:
#use sdpc to connect to the ztw-restful-cloud application

# generate the network connection script
from getpass import getpass
import subprocess

password = getpass()
script_content = f"""
#!/bin/bash
echo {password} | sudo -S /opt/homebrew/bin/sdpc -apikey {apikey} -sdp_server https://stage.ztw.splashtop.com -org demo  {application_id}
"""

# Write the script_content to a file
with open('script.sh', 'w') as f:
    f.write(script_content)

# Change the permission of the script to be executable
!chmod +x script.sh

# Run the script in a subprocess, if the connection established, you should see the output: ztw-restful-cloud connect success 
subprocess.Popen(['./script.sh'],  shell=True)


<Popen: returncode: None args: ['./script.sh']>

Password:

ztw-restful-cloud connect success 


## API Call to The Secure Workspace Managed Remote Application
With the network tunneling established to the remote restful application, we will invoke the API from this notebook. The example application is a simple restful API that returns: 

```json
{
    "message": "Hello World!"
}
```

In [133]:
headers = {"Accept": "application/json, text/plain, */*", "Authorization": f"Bearer {sdp_token_new}", "Content-Type": "application/json;charset=UTF-8"}
response = requests.get(f"http://ztenv-myrestful.default.svc.cluster.local:8080/", headers=headers)

if response.status_code == 200 :
    print (f"Success: {response.json()} \n")
    
else: 
    ERROR_MESSAGE += f"Error: HTTP {response.status_code} - {response.text} \n"
    print (ERROR_MESSAGE)


Success: {'message': 'HelloWorld'} 



## Clean Up and Kill the Tunnel

In this cell we will clean up the application and kill the tunnel. The script will prompt for your local os password in order to run sudo, then you should wait for the cli to kill the tunnel.

In [135]:
# kill the process to disconnect the secure workspace
!echo {getpass()} | sudo -S killall sdpc
!rm -f script.sh

Password:No matching processes were found
